# Importing Libraries

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import math
import pandas as pd
import json

## Loading Webscraper Tools

In [2]:
# Path to chromedriver
!which chromedriver

'which' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# Windows users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Loading Webpage

In [4]:
# Visit the Bootcamp website to Scrape
url = 'https://www.redwoodcity.org/departments/city-manager/news-room/news-releases'
browser.visit(url)

In [5]:
# Parse the HTML at the News page (Only)
html = browser.html
newspagesoup = BeautifulSoup(html, 'html.parser')

## Waiting for browser to webpage

## Controlling Webpage

##### [Variables for News Content Set-Up]

In [6]:
# News Data Variable Lists:
newsTitle = []
newsPostDetail = []
newsContent = []

##### [City News Table of Contents: Key root values]

In [7]:
# Obtaining city url, so we can compare with blog url as part of the test:
# Finding index where URL ends with 'org'
orgCheck = browser.url.find('org')
# Saving city URL
redwoodCityUrl = browser.url[:orgCheck+3]
# City News table of contents limiter
redwoodCityWidgetLimiter = len(newspagesoup.find_all('div', attrs={"class": "content_area normal_content_area clearfix"})[0].get_text())

##### [Page Turn Set-up]

In [8]:
# - Inspecting webpage to get the ['##'] of '##'
# Inspects webpage for total data loaded:
x1 = browser.find_by_css('.pager-info').text.find('-')
x2 = browser.find_by_css('.pager-info').text.find('of')
dataLoadedStep = int(browser.find_by_css('.pager-info').text[x1+1:x2].strip()) 

# - Inspecting webpage to get the '##' of ['##']
# Checking to see if there are more pages to get data from:
x3 = browser.find_by_css('.pager-info').text.find('of')
x4 = browser.find_by_css('.pager-info').text.find('items')
dataLoadedMax = int(browser.find_by_css('.pager-info').text[x3+2:x4].strip())

# Turn page x number of times:
pageTurn = math.ceil(dataLoadedMax/dataLoadedStep)

##### [News links which cause users to open a blank tab]

In [9]:
def findTargetBlank():
    
    print('\nStarting Function to find Finding Target _Blank links:\n')
    
    html = browser.html
    newspagesoup = BeautifulSoup(html, 'html.parser')
    
    global newWindowTabTest, newWindowTab, dataLoadedState
    
    # Method 1: This prints the out our results
    # Locating news headline which triggers a new window cmd (target="_blank") only:
    newsHeadlinesBlankWindow = newspagesoup.find_all('a', attrs={"class": "item-title", "target":"_blank"})
    print(f'\nMethod 1: Element(s) contents \n {newsHeadlinesBlankWindow}\n')

    # locating row # in which link will be located during the main loop:
    # Turning element(s) to string for workability purpose
    newsHeadlinesBlankWindow = str(newsHeadlinesBlankWindow)
    # Starting Index # for '...target="_blank">'
    startingIndex = newsHeadlinesBlankWindow.find('target')
    # Ending Index # for '...target="_blank">'
    endingIndex = newsHeadlinesBlankWindow.find('>')
    # Condition for which news link will open a blank window
    newWindowTabTest = newsHeadlinesBlankWindow[startingIndex:endingIndex] == 'target="_blank"'
    print(f'\nNew Window/Tab Case: {newWindowTabTest}\n')
    
    print('# -------------------------------------------------------------------------------------------------------- #')

    # ----------------------------------------------------------------------------------------------------------------- #
    
    if newWindowTabTest == True:
        # Method 2: Uses a temp. DataFrame to find which element containes [target="_blank"]
        # - Case only accounts for one link currently future upgrade will be for multiple links
        # newWindowTab = []
        # newWindowTabclick = []

        # - Inspecting webpage to get the ['##'] of '##'
        x1 = browser.find_by_css('.pager-info').text.find('-')
        x2 = browser.find_by_css('.pager-info').text.find('of')
        dataLoadedState = int(browser.find_by_css('.pager-info').text[x1+1:x2].strip())

        # - Comparing ['##'] of ['##'] to determine if still more data available
        if dataLoadedState != dataLoadedMax:
            print('\nStill more data to load\n')
        else:
            print('\nFinal set of data loading\n')

        # Identify all news link types (using beautiful soup):
        newsHeadlines = newspagesoup.find_all('a', class_='item-title')

        # Putting Data in temp. DataFrame helps with organizing data & finding correct row #:
        newWindowTabDF = pd.DataFrame({'a-tags':newsHeadlines})
        
        # If statement: For total number of new headlines loaded on page equal to the step-value then range=step
        if dataLoadedState%dataLoadedStep == 0:
            # Scanning news links which contain target="_blank":
            for i in range(0,dataLoadedState):
                # If value is greater then 0 then that means that string contains [target="_blank"]    
                if str(newWindowTabDF.iloc[i,0]).find('target="_blank"') > 0:
                    # Store index i value into variable, so we can reference it as special condition
                    newWindowTab = i
                    newWindowTabClick = browser.find_by_css('.list-main').find_by_tag('a')[newWindowTab]
        # Else: Range will change from 0 to remainder-value
        else:
            # New range: 0 to remainder-value
            # Scanning news links which contain target="_blank":
            for i in range(0,dataLoadedState%dataLoadedStep):
                # If value is greater then 0 then that means that string contains [target="_blank"]    
                if str(newWindowTabDF.iloc[i,0]).find('target="_blank"') > 0:
                    # Store index i value into variable, so we can reference it as special condition
                    newWindowTab = i
                    newWindowTabClick = browser.find_by_css('.list-main').find_by_tag('a')[newWindowTab]
        # News links which will cause us to open a new window during our data extraction process
        print(f'\nMethod 2: New Window/Tab Case at Index(s): #{newWindowTab}\n')
    else:
        print('\nCase not present\n')

##### [News links which cause users to open a blank tab]

In [10]:
def Tests():
    
    global i

#T1 # Check if current i-value will open a new window/tab:
    if ((i == newWindowTab) and (newWindowTabTest == True)):
        print('\nNew Window case:')
        # - Click open news headline(s)
        browser.find_by_css('.list-main').find_by_tag('a')[i].click()
        # - Making opened paged as current window
        window = browser.windows[1]
        window.is_current = True
        # - Store Data in variable list:
        document = browser.url
        newsContent.append(f"Document URL: {document}")
        # - Closing new window
        window = browser.windows[1]
        window.close()
        # - Making default page current page again
        window = browser.windows[0]
        window.is_current = True
        newsTitle.append(browser.find_by_css('.list-main').find_by_tag('a')[i].text)
        newsPostDetail.append(browser.find_by_css('.list-main').find_by_tag('p')[i].text)
    else:
        # Clicking on news header element:
        browser.find_by_css('.list-main').find_by_tag('a')[i].click()
        
# T2    # Test for Blog:
        # Check if URL has changed to Blog URL
        comCheck = browser.url.find('com')
        mediumBlog = browser.url[:comCheck+3]

        if mediumBlog.find('medium') >= 0:
            print('Blog Case:')
            # Prints 'True' when url changes we need to scrape the blog page with new code here:
            # Appending content data into newsContent variable:
            html = browser.html
            newspagesouplinks = BeautifulSoup(html, 'html.parser')
            # Flaw in line but okay for this use:
            # - Initial news table of contents contains 1 widget with 48 characters.
            # - Other pages which contain widget typically contain more than 48 characters.
            blogContent = newspagesouplinks.find_all('section')[1].get_text()
            blogUrl = browser.url
            # Telling browser to go back to news list page:
            browser.back()
            # Going back a page we check to see if data has loaded.
            browser.is_element_present_by_css('.list-main', wait_time=5)
            # Appending data into data variables
            newsTitle.append(browser.find_by_css('.list-main').find_by_tag('a')[i].text)
            newsPostDetail.append(browser.find_by_css('.list-main').find_by_tag('p')[i].text)
            newsContent.append(f"{blogContent} Blog URL: {blogUrl}")
        else:
# T3        # Test for Regular Post w/ Widgets:
            html = browser.html
            newspagesouplinks = BeautifulSoup(html, 'html.parser')
            # Flaw in line but okay for this use:
            # - Initial news table of contents contains 1 widget with 48 characters.
            # - Other pages which contain widget typically contain more than 48 characters.
            widgetCheckOnPage = newspagesouplinks.find_all('div', attrs={"class": "content_area normal_content_area clearfix"})
            if widgetCheckOnPage == []:
                if browser.is_element_present_by_css('.detail-title') == 1:
# T4                # Test for Regular Post:
                    # Test Requirements for Regular Post:
                    print('\nRegular Post Case:')
                    print(browser.is_element_present_by_css('.detail-title') == 1)
                    print(browser.is_element_present_by_css('.detail-list') == 1)
                    print(browser.is_element_present_by_css('.detail-content') == 1)
                    # Appending data into data variables:
                    newsTitle.append(browser.find_by_css('.detail-title').text)
                    newsPostDetail.append(browser.find_by_css('.detail-list').text)
                    newsContent.append(browser.find_by_css('.detail-content').text)
                    # Telling browser to go back to news list page:
                    browser.back()
                else:
                    print(f'\nunkown content in news link for new row {i}')
                    # Telling browser to go back to news list page:
                    browser.back()
            else:
                print('\n Regular Post w/ Widget Case:')
                widgetCheck = len(newspagesouplinks.find_all('div', attrs={"class": "content_area normal_content_area clearfix"})[0].get_text())
                # Check if element is present
                if widgetCheck > redwoodCityWidgetLimiter:
                    #
                    widgetlink = browser.url
                    # Telling browser to go back to news list page:
                    browser.back()
                    newsTitle.append(browser.find_by_css('.list-main').find_by_tag('a')[i].text)
                    newsPostDetail.append(browser.find_by_css('.list-main').find_by_tag('p')[i].text)
                    newsContent.append(widgetlink)
                else:
                    print(f'\nunkown content in news link for new row {i}')
                    # Telling browser to go back to news list page:
                    browser.back()

##### [Data Mining Loop]

In [11]:
# Placing Main loop into a function
def scrapeDataLoop():
    
    global dataLoadedState, x1, x2, i
    
    print('\nStarting Scraping Function:\n')
    
    # - Inspecting webpage to get the ['##'] of '##'
    # Inspects webpage for total data loaded:
    x1 = browser.find_by_css('.pager-info').text.find('-')
    x2 = browser.find_by_css('.pager-info').text.find('of')
    dataLoadedState = int(browser.find_by_css('.pager-info').text[x1+1:x2].strip())
    
    # If statement: For total number of new headlines loaded on page equal to the step-value then range=step
    if dataLoadedState%dataLoadedStep == 0:
        print(f'\nNumber of news headlines loaded in page: {dataLoadedStep}\n')
    else:
        print(f'\nNumber of news headlines loaded in page: {dataLoadedState%dataLoadedStep}\n')
    
    # If statement: For total number of new headlines loaded on page equal to the step-value then range=step
    if dataLoadedState%dataLoadedStep == 0:
        # For loop to open news headline, collect content, go back & repeat:
        for i in range(0,dataLoadedState):
            Tests()
        
    # Else: Range will change from 0 to remainder-value
    else:
        # For loop to open news headline, collect content, go back & repeat:
        for i in range(0,dataLoadedState%dataLoadedStep):
            Tests()

##### Data Collection Code

In [12]:
startPageTurnValue = pageTurn-1

# 1. Main Loop [Open each news link on page]:
for i in range(startPageTurnValue):
    if i == 0:
        print(f'page {i+1}')
        #
        findTargetBlank()
        # Calling function to scrape data
        scrapeDataLoop()
        # Loading next page
        browser.find_by_css('.pg-next-button').click()
        #
        print(f'page {i+2}')
        findTargetBlank()
        # Calling function to scrape data
        scrapeDataLoop()
        # Loading next page
        
    else:
        print(f'page {i}')
        #
        browser.find_by_css('.pg-next-button').click()
        #
        findTargetBlank()
        # Calling function to scrape data
        scrapeDataLoop()
        # Loading next page

page 1

Starting Function to find Finding Target _Blank links:


Method 1: Element(s) contents 
 [<a class="item-title" href="/Home/Components/News/News/3468/874" target="_blank">Notice of Election</a>]


New Window/Tab Case: True

# -------------------------------------------------------------------------------------------------------- #

Still more data to load


Method 2: New Window/Tab Case at Index(s): #4


Starting Scraping Function:


Number of news headlines loaded in page: 20


Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

New Window case:

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True
Blog Case:

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True
True
True

Regular Post Case:
True


## Printing Data Results

###### Checking to make sure all lists have the same length

In [13]:
if len(newsTitle) == len(newsPostDetail) == len(newsContent):
    print(f'Lists match in length: {len(newsTitle)}')
else:
    print(f'Error lists are not matching in length {len(newsTitle)}, {len(newsPostDetail)}, {len(newsContent)}')

Lists match in length: 33


### Cleaning Content List Data

###### Content List Cleaned/Parsed Data

In [40]:
# Creating DataFrame and removing unwanted text
df = pd.DataFrame({'Title':newsTitle, 'Details':newsPostDetail, 'Content':newsContent}).replace({r'Post Date:': '', r'\n': ' ',}, regex=True)

In [38]:
for i in range(0,len(newsTitle)-1):
    # replaces [""|''|etc.] with escape characters required for javascript to read.
    df.iloc[i,2] = json.dumps(df.iloc[:,2][i])
df

,Title,Details,Content
0,Redwood City City Council to Consider Revised ...,10/23/2020 11:32 AM,"""At their regularly scheduled meeting Monday, ..."
1,Join us for STEAM at the Library throughout Oc...,10/12/2020 1:33 PM,"""Redwood City, California \u2013 The Redwood C..."
2,Redwood City is Listening: Update on on Racial...,10/01/2020 3:34 PM,"""Over the last several months, we have been ac..."
3,News Release - Redwood City to Hold First Elec...,07/01/2020 4:45 AM,"""FOR IMMEDIATE RELEASE Media Contact: Jennifer..."
4,Notice of Election,06/26/2020 5:12 PM,"""Document URL: https://www.redwoodcity.org/Hom..."
5,News Release - Fourth of July Celebration in R...,06/25/2020 4:45 AM,"""FOR IMMEDIATE RELEASE Media Contact: Jennifer..."
6,News Release - City Council to Consider Recomm...,06/19/2020 1:43 PM,"""FOR IMMEDIATE RELEASE June 19, 2020 Media Con..."
7,Fiscal Year 2020-21 Recommended Budget (Blog P...,06/09/2020 11:59 AM,"""Fiscal Year 2020\u201321 Recommended Budgetso..."
8,Community Message from Mayor Howard and Chief ...,06/05/2020 9:31 PM,"""Earlier this year, the City Council set our v..."
9,Message to the Community,06/01/2020 5:24 PM,"""To our Redwood City Community, We have becom..."


### Scrap Code

In [43]:
for i in range(0,32):
    # replaces [""|''|etc.] with escape characters required for javascript to read.
    json.dumps(df.iloc[:,2][i])
    print(f'Element {i}: \n{json.dumps(df.iloc[:,2][i])}\n')
    # Format to copy and paste onto javascript:
#     print(f'{json.dumps(df.iloc[:,2][i])},')

Element 0: 
"At their regularly scheduled meeting Monday, October 26, at 7 p.m., the Redwood City City Council will consider a Revised Fiscal Year 2020-21 Budget, which proposes a reduction in expenditures of $7.7 million from the Fiscal Year 2020-21 Adopted Budget. City operating revenues are expected to drop 8.1% or $13.1 million compared to Fiscal Year 2019-20 and the City expects that it will take at least five years for revenue to return to the Fiscal Year 2019-20 levels. When reduced revenues are combined with known cost increases to fund pension and retiree health obligations over the next 10 years, the City must take action to reduce operating costs and seek new revenues. Without action, the City faces annual deficits of $7-13 million per year for the foreseeable future.  \u201cThe Revised Budget should be thought of as a transitional budget while staff analyze current services and assess opportunities to reduce costs and modify service delivery approaches. We have work to do, 